<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Entrenamiento/Entrenamiento_YoloV8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Training YOLO v8 model for Nuclei segmentation in H&E-Stained Images** <br>

Train custom instance segmentation model using YOLOv8 - on your own dataset.
<p>
This is part 2 of the two part tutorial on training custom nuclei model where we have used Detectron2 (Mask R-CNN) is part 1 and here we will use YOLO v8 for instance segmentation.
<p>

This code is a complete project, with the following steps <br>

* Training YOLOv8 model
* Testing the trained model on test dataset
* Extracting morphological measurements from the detected objects
* Plotting results

<p>

But first make sure you get your data ready for training. Please watch my video tutorial associated with this code to learn about downloading labeled masks and converting them to COCO JSON format and then to YOLO v8 format.

<p>

**Nuclei dataset can be downloaded from:**https://www.kaggle.com/datasets/ipateam/nuinsseg?resource=download
<br>

**Dataset description:** https://arxiv.org/abs/2308.01760
<br>
<p>

**Summary of the dataset:** <p>
The NuInsSeg dataset contains more than *30k manually segmented nuclei from 31 human and mouse organs and 665 image patches extracted from H&E-stained whole slide images*. We also provide ambiguous area masks for the entire dataset to show in which areas manual semantic/instance segmentation were impossible.
<p>

**Human organs:**

cerebellum, cerebrum (brain), colon (rectum), epiglottis, jejunum, kidney, liver, lung, melanoma, muscle, oesophagus, palatine tonsil, pancreas, peritoneum, placenta, salivary gland, spleen, stomach (cardia), stomach (pylorus), testis, tongue, umbilical cord, and urinary bladder
<p>

**Mouse organs:**

cerebellum, cerebrum, colon, epiglottis, lung, melanoma, muscle, peritoneum, stomach (cardia), stomach (pylorus), testis, umbilical cord, and urinary bladder)

**Install the required libraries:**

Let us start by installing ultralytics library. All other libraries should be pre-installed on colab. If you are working on a local system, please make sure you install matplotlib, Pillow, numpy, Seaborn, and roboflow. You may also want to install pandas and other libraries depending on the task.

In [ ]:
# Install the ultralytics package using pip
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np

def load_random_image_and_labels(root_dir, dataset_type):
    image_dir = os.path.join(root_dir, dataset_type, 'images')
    label_dir = os.path.join(root_dir, dataset_type, 'labels')

    images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    random_image_name = random.choice(images)

    image_path = os.path.join(image_dir, random_image_name)
    label_path = os.path.join(label_dir, random_image_name.replace('.jpg', '.txt'))

    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {image_path}")

    labels = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = [float(x) for x in parts[1:5]]
            segmentation = [float(x) for x in parts[5:]]
            labels.append((class_id, bbox, segmentation))

    return image, labels

def draw_labels(image, labels):
    height, width = image.shape[:2]

    for label in labels:
        class_id, bbox, segmentation = label

        # Convert bounding box from relative to absolute coordinates
        x_center, y_center, w, h = bbox
        x_center *= width
        y_center *= height
        w *= width
        h *= height

        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw segmentation points
        if segmentation:
            points = np.array(segmentation).reshape(-1, 2)
            points[:, 0] *= width
            points[:, 1] *= height
            points = points.astype(np.int32)
            cv2.polylines(image, [points], isClosed=True, color=(255, 0, 0), thickness=2)

    return image

def visualize_dataset(root_dir, dataset_types):
    for dataset_type in dataset_types:
        image, labels = load_random_image_and_labels(root_dir, dataset_type)
        image_with_labels = draw_labels(image, labels)

        plt.figure(figsize=(10, 10))
        plt.title(f"{dataset_type.capitalize()} - Imagen con Etiquetas")
        plt.imshow(cv2.cvtColor(image_with_labels, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()


# Ruta raíz del dataset
root_dir = '/content/drive/MyDrive/TFM/Sin duplicados/Yolo&Coco/All_Data_Augmentation'

# Visualizar aleatoriamente una imagen y etiquetas para train, valid y test
dataset_types = ['train',  'train',  'train',  'train',  'train',  'valid', 'test']
visualize_dataset(root_dir, dataset_types)


**Import a model and populate it with pre-trained weights.**
<p>
Here, we are importing an instance segmentation model with weights. For a list of pre-trained models, checkout: https://docs.ultralytics.com/models/yolov8/#key-features

In [ ]:
#Instance
model = YOLO('yolov8n-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8n-seg.pt')  # Transfer the weights from a pretrained model (recommended for training)

100%|██████████| 6.74M/6.74M [00:00<00:00, 352MB/s]


On colab, you may encounter encoding issues while working with certain libraries (e.g., installing roboflow) so let's go ahead and run the following cell.  

In [ ]:
#Withut this Colab is giving an error when installing Roboflow
import locale
import os
locale.getpreferredencoding = lambda: "UTF-8"

Let us load the YAML file that contains the names of our classes, number of classes and the directories for train, valid, and test datasets, respectively.

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat /content/drive/MyDrive/TFM/dataset_Detectron2_V3/data.yaml

train: ../train/images
val: ../val/images
test: ../test/images

nc: 1
names: ['Polipo']

roboflow:
  workspace: master-c9yad
  project: polipos
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/master-c9yad/polipos/dataset/1

In [ ]:
# define number of classes based on YAML
import yaml
with open("/content/drive/MyDrive/TFM/dataset_Detectron2_V3/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

**Train the model**

In [ ]:
ruta_raiz = "/content/drive/MyDrive/TFM/models/YoloV8_Models/results/"

In [ ]:
#Define a project --> Destination directory for all results
project = ruta_raiz
#Define subdirectory for this specific training
name = "250_epochs-" #note that if you run the training again, it creates a directory: 3_epochs-2
ruta_modelo = os.path.join(ruta_raiz, name)

In [ ]:
# Train the model
results = model.train(data='/content/drive/MyDrive/TFM/dataset_Detectron2_V3/data.yaml',
                      project=project,
                      name=name,
                      epochs=250,
                      patience=0, # I am setting patience=0 to disable early stopping.
                      batch=4,
                      imgsz=512)

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/drive/MyDrive/TFM/dataset_Detectron2_V3/data.yaml, epochs=250, time=None, patience=0, batch=4, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/TFM/models/YoloV8_Models/results/, name=250_epochs-, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

train: Scanning /content/drive/MyDrive/TFM/dataset_Detectron2_V3/train/labels.cache... 143 images, 0 backgrounds, 0 corrupt: 100%|██████████| 143/143 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/TFM/dataset_Detectron2_V3/val/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/TFM/models/YoloV8_Models/results/250_epochs-/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/TFM/models/YoloV8_Models/results/250_epochs-
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    237/250     0.671G     0.4438     0.6261     0.4038     0.9358          7        512: 100%|██████████| 36/36 [00:07<00:00,  5.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  5.40it/s]

                   all         49         51      0.815      0.706      0.772      0.569      0.815      0.706      0.782      0.588



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    238/250     0.669G     0.3525     0.5822     0.3667     0.8747          9        512:  22%|██▏       | 8/36 [00:02<00:08,  3.38it/s]

In [ ]:
# Start tensorboard
# Launch after you have started training
# # %reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir ruta_modelo --port 6007

All training curves, metrics, and other results are stored as images in the 'runs' directory. Let us open a couple of these images. <p>
Please note that from here on I will be working the model that I've already trained for 50 epochs.

In [ ]:
from IPython.display import Image

In [ ]:
ruta_imagen = os.path.join(ruta_modelo, "results.png")
Image(filename=ruta_imagen)


In [ ]:
ruta_train_batch = os.path.join(ruta_modelo, "train_batch2.jpg")
Image(filename=ruta_train_batch, width=900)

**Run inference**

Now that our model is trained, we can use it for inference.

In [ ]:
import glob

#List the saved models in 'runs' directory. Note that you will see multiple 'train' subdirectories numbered 1, 2, 3, etc. The exact number depends on the number of epochs.
ruta_weights = os.path.join(ruta_modelo, "weights")

# Usar glob para obtener la lista de archivos en la ruta especificada
archivos_en_ruta = glob.glob(ruta_weights + "/*")

# Mostrar los archivos encontrados
for archivo in archivos_en_ruta:
    print(archivo)

/content/drive/MyDrive/TFM/models/YoloV8_Models/results/3_epochs-/weights/last.pt
/content/drive/MyDrive/TFM/models/YoloV8_Models/results/3_epochs-/weights/best.pt


You can load the best model or the latest. I am picking the latest.

In [ ]:
ruta_last_weight = os.path.join(ruta_weights, "last.pt")

my_new_model = YOLO(ruta_last_weight)

Load an image and perform inference (segmentation).

In [ ]:
import random
# Directorio que contiene las imágenes de prueba
ruta_imagenes = "/content/drive/MyDrive/TFM/dataset_Detectron2_V3/test/images"

# Obtener la lista de nombres de archivo en el directorio
archivos = os.listdir(ruta_imagenes)

# Elegir un nombre de archivo al azar
nombre_archivo = random.choice(archivos)

# Ruta completa de la imagen seleccionada
ruta_imagen = os.path.join(ruta_imagenes, nombre_archivo)

# Hacer la inferencia en la imagen seleccionada
new_results = my_new_model.predict(ruta_imagen, conf=0.5)  # Ajusta el umbral de confianza según sea necesario

In [ ]:
new_image = '/content/drive/MyDrive/ColabNotebooks/data/NuInsSeg_Nuclei_dataset/yolo_dataset/test/images/human_liver_22.png'
new_results = my_new_model.predict(new_image, conf=0.5)  #Adjust conf threshold


The results are stored in a variable 'new_results'. Since we only have one image for segmentation, we will only have one set of results. Therefore, let us work with that one result.

In [ ]:
# Suponiendo que new_results[0] es una instancia de algún objeto que tiene un método plot() que devuelve la imagen
new_result_array = new_results[0].plot()

# Si new_result_array es el resultado de la función plot(), y es una imagen en formato RGB o BGR

# Si la imagen está en formato BGR, es posible que necesite convertirla a RGB antes de mostrarla con matplotlib
new_result_array_rgb = new_result_array[:, :, ::-1]  # Esta línea invierte el orden de los canales de color (BGR a RGB)

# Crear una figura de matplotlib y mostrar la imagen
plt.figure(figsize=(9, 9))
plt.imshow(new_result_array_rgb)
plt.axis('off')  # Desactivar los ejes si no son necesarios

# Mostrar la figura
plt.show()


# Segmenting and analyzing multiple images

Now, let us segment all our test images, perform measurements for all objects, capture them into a pandas dataframe and save the dataframe into a csv file.

In [ ]:
import os
import csv
import cv2
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops

# Directory path to the input images folder
input_images_directory = "/content/drive/MyDrive/ColabNotebooks/data/NuInsSeg_Nuclei_dataset/yolo_dataset/test/images"

# Output directory where the CSV file will be saved
output_csv_path = "/content/drive/MyDrive/ColabNotebooks/data/NuInsSeg_Nuclei_dataset/yolo_dataset/test_results/output_objects_yolo.csv"

# Extract the directory name from the full path
output_dir_name = os.path.dirname(output_csv_path)

# Check if the directory exists
if not os.path.exists(output_dir_name):
    os.makedirs(output_dir_name)

# List of valid image extensions. This ensures that the code doesn't throw
# errors if your directory has non-images, like .json or other text files.
valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif']

# Open the CSV file for writing
with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row in the CSV file
    csvwriter.writerow(["File Name", "Class Name", "Object Number", "Area", "Centroid", "BoundingBox"])

    # Loop over the images in the input folder
    for image_filename in os.listdir(input_images_directory):
        # Check if the file has a valid image extension
        if not any(image_filename.lower().endswith(ext) for ext in valid_extensions):
            continue

        image_path = os.path.join(input_images_directory, image_filename)
        new_im = cv2.imread(image_path)

        # Perform prediction on the new image
        new_results = my_new_model.predict(new_im, conf=0.2)  # Adjust conf threshold


        # Access the bounding boxes and class labels from new_results
        bounding_boxes = new_results[0].boxes.data.cpu().numpy()  # Move to CPU and convert to NumPy array
        class_labels = [0 for _ in range(len(bounding_boxes))]  # Assuming all objects are 'Nuclei'

        # Write the object-level information to the CSV file
        for i, bbox in enumerate(bounding_boxes):
            object_number = i + 1
            x1, y1, x2, y2 = bbox[:4]  # Only take the first 4 values
            area = (x2 - x1) * (y2 - y1)
            centroid = ((x1 + x2) / 2, (y1 + y2) / 2)
            bounding_box = (x1, y1, x2, y2)

            #
            class_name = 'Nuclei'  # Since all objects are 'Nuclei' in this example

            csvwriter.writerow([image_filename, class_name, object_number, area, centroid, bounding_box])

print("Object-level information saved to CSV file.")



0: 512x512 58 Nucleis, 15.4ms
Speed: 3.7ms preprocess, 15.4ms inference, 4.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 38 Nucleis, 10.4ms
Speed: 2.0ms preprocess, 10.4ms inference, 3.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 112 Nucleis, 10.3ms
Speed: 2.3ms preprocess, 10.3ms inference, 4.8ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 47 Nucleis, 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 3.4ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 7 Nucleis, 8.1ms
Speed: 3.2ms preprocess, 8.1ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 15 Nucleis, 9.4ms
Speed: 2.1ms preprocess, 9.4ms inference, 2.8ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 51 Nucleis, 11.7ms
Speed: 1.5ms preprocess, 11.7ms inference, 3.3ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 39 Nucleis, 8.9ms
Speed: 2.2ms preprocess, 8.9ms inference, 4.4ms postprocess per image at shape

Object-level information saved to CSV file.
